In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/data/public/MLA/VCHAMPS-Test/inpatient_specialty_test.csv').iloc[:,1:]

In [3]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [4]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

/tmp/ipykernel_219886/2218735275.py:2: RuntimeWarning: Mean of empty slice
  period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))


In [5]:
period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [6]:
period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

In [7]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [8]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

In [9]:
specialty_count = inpatient_specialty.groupby('Internalpatientid')['Specialty'].count().reset_index()
specialty_count = specialty_count.rename(columns={'Specialty': 'specialty count'})

specialty_medical_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(0).sum()).reset_index()
specialty_support_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(1).sum()).reset_index()

specialty_medical_count = specialty_medical_count.rename(columns={'Specialty encoded': 'specialty medical count'})
specialty_support_count = specialty_support_count.rename(columns={'Specialty encoded': 'specialty support count'})

In [10]:
age_mean = inpatient_specialty.groupby('Internalpatientid')['Age at specialty'].mean().reset_index()
age_mean = age_mean.rename(columns={'Age at specialty': 'mean age at specailty'})

In [11]:
listTables = [period_mean, period_std, specialty_medical_count,specialty_support_count, specialty_count ]
inpatient_specialty_preprocessed = age_mean.copy()
for i in listTables:
    inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.merge(i,how = 'left', on = 'Internalpatientid')


In [12]:
inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.dropna(subset=['period mean'])

In [14]:
#inpatient_specialty_preprocessed.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_preprocessed_test.csv')